MOVIE RECOMMENDER SYSTEM

Importing the dependencies

In [155]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and pre-Processing

In [156]:
#loading the data from the csv file to pandas dataframe
movies_data= pd.read_csv('movies.csv')

In [157]:
#printing the first 5 data
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [158]:
#number of rows and coloumns
movies_data.shape

(4803, 24)

In [159]:
#selecting the relevant features for recommendation
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [160]:
#repacing the missing values with null string
for feature in selected_features:
    movies_data[feature]= movies_data[feature].fillna('')


In [161]:
#combining all the 5 selectes features
combined_features = movies_data['genres']+movies_data['keywords']+movies_data['cast']+movies_data['tagline']+movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceEdward Burns Kerry Bish\u00e9 Ma...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [162]:
#converting the text data into the feature vertex
vectorizer=TfidfVectorizer()

In [163]:
feature_vectors= vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 111730 stored elements and shape (4803, 27793)>
  Coords	Values
  (0, 249)	0.0787885735193979
  (0, 449)	0.09132857870544613
  (0, 9006)	0.11836592141063045
  (0, 22021)	0.09998166261909516
  (0, 9381)	0.27456446600852247
  (0, 4480)	0.21855919626397677
  (0, 10295)	0.16575087043546832
  (0, 23106)	0.337556237272303
  (0, 26544)	0.13093417905590005
  (0, 4701)	0.2459238418151712
  (0, 22971)	0.27456446600852247
  (0, 27391)	0.23540668329185652
  (0, 27753)	0.1996131269037809
  (0, 21668)	0.2160572904494248
  (0, 22654)	0.2160572904494248
  (0, 26765)	0.19689340044337153
  (0, 23464)	0.15455418218914943
  (0, 14664)	0.22425804599456597
  (0, 16741)	0.1596538352412605
  (0, 20901)	0.27456446600852247
  (0, 24390)	0.07537942267634702
  (0, 27372)	0.12652446609697676
  (0, 18317)	0.08713254643393821
  (0, 18714)	0.2618907879051872
  (0, 13347)	0.116223484793762
  :	:
  (4801, 18945)	0.22049236993953694
  (4801, 11877)	0.29519313

Cosine Similarity

In [164]:
#getting the similarity score using cosine similarity
similarity=cosine_similarity(feature_vectors)

In [165]:
print(similarity.shape)

(4803, 4803)


In [166]:
#getting the movie name from user
movie_name =input("Enter your favoite movie name:")


In [167]:
#creating a list with all the movie names given in the dataset
list_of_titles =movies_data['title'].tolist()
print(list_of_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [168]:
#finding the close match for the movie name give y the user
close_match=difflib.get_close_matches(movie_name,list_of_titles)
print(close_match)

['The Avengers', 'Swingers']


In [169]:
closest_match =close_match[0]
print(closest_match)


The Avengers


In [170]:
#finding the index of the movie with title
index_movie=movies_data[movies_data.title == closest_match]['index'].values[0]
print(index_movie)

16


In [171]:
#getting a list of similar movies
similarity_score = list(enumerate(similarity[index_movie]))
print(similarity_score)


[(0, np.float64(0.015170921791145934)), (1, np.float64(0.0)), (2, np.float64(0.032525905381778845)), (3, np.float64(0.006351604344857927)), (4, np.float64(0.024331760362412226)), (5, np.float64(0.005987994503869023)), (6, np.float64(0.0)), (7, np.float64(0.5880344072861116)), (8, np.float64(0.0)), (9, np.float64(0.12698253490823463)), (10, np.float64(0.017455079954331765)), (11, np.float64(0.005360407736542601)), (12, np.float64(0.0)), (13, np.float64(0.005616934073061768)), (14, np.float64(0.13600155226945232)), (15, np.float64(0.009424200156071743)), (16, np.float64(1.0000000000000002)), (17, np.float64(0.005692377961345938)), (18, np.float64(0.014682313976692006)), (19, np.float64(0.005128609195506847)), (20, np.float64(0.09647378954468569)), (21, np.float64(0.0218792330876644)), (22, np.float64(0.0)), (23, np.float64(0.03138810843269163)), (24, np.float64(0.0)), (25, np.float64(0.009976814009058114)), (26, np.float64(0.2508944539495725)), (27, np.float64(0.013668637146892267)), (28

In [172]:
len(similarity_score)

4803

In [173]:
#sorting the movies based on similarity score
sorted_similarity_movies = sorted(similarity_score, key=lambda x:x[1],reverse =True)
print(sorted_similarity_movies) 

[(16, np.float64(1.0000000000000002)), (7, np.float64(0.5880344072861116)), (79, np.float64(0.288288603045853)), (174, np.float64(0.28368684711085745)), (85, np.float64(0.2532296181631358)), (26, np.float64(0.2508944539495725)), (511, np.float64(0.240197852871552)), (64, np.float64(0.2248007999329226)), (203, np.float64(0.22030038717361208)), (126, np.float64(0.21250331366904007)), (33, np.float64(0.2109859115040518)), (788, np.float64(0.20562124350049354)), (182, np.float64(0.20040198486976962)), (129, np.float64(0.18815585435212273)), (38, np.float64(0.18760827003775962)), (46, np.float64(0.18203639620651485)), (68, np.float64(0.16749071872686902)), (4401, np.float64(0.15473824032021816)), (1294, np.float64(0.1381211179434387)), (14, np.float64(0.13600155226945232)), (870, np.float64(0.1326701883810275)), (1740, np.float64(0.1308741102487735)), (101, np.float64(0.1305444721128171)), (782, np.float64(0.129865294967572)), (9, np.float64(0.12698253490823463)), (1720, np.float64(0.115308

In [176]:
#print the name of similar movies based on the index
print('movies suggested for u:\n')
i=1
for movie in sorted_similarity_movies:
    index=movie[0]
    title_from_index=movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1



movies suggested for u:

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Iron Man 2
4 . The Incredible Hulk
5 . Captain America: The Winter Soldier
6 . Captain America: Civil War
7 . X-Men
8 . X-Men: Apocalypse
9 . X2
10 . Thor: The Dark World
11 . X-Men: The Last Stand
12 . Deadpool
13 . Ant-Man
14 . Thor
15 . The Amazing Spider-Man 2
16 . X-Men: Days of Future Past
17 . Iron Man
18 . The Helix... Loaded
19 . Serenity
20 . Man of Steel
21 . Superman II
22 . Kick-Ass 2
23 . X-Men: First Class
24 . The Spirit
25 . Batman v Superman: Dawn of Justice
26 . Kick-Ass
27 . Zodiac
28 . Iron Man 3
29 . A Scanner Darkly


Movie Recommendation Sytem
